<a href="https://colab.research.google.com/github/lala13al/-Cibe/blob/main/prototipo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import requests
import json
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Obtener precios de cierre
symbol = "BTCUSDT"
interval = "1m"
start_time = "2020-01-01T00:00:00Z" #Cambiamos las fechas para obtener mas datos históricos
end_time = "2022-12-31T00:00:00Z"

# Convert start_time and end_time to UNIX timestamp
start_time = int((datetime.strptime("2020-01-01T00:00:00Z", '%Y-%m-%dT%H:%M:%SZ') - datetime(1970,1,1)).total_seconds()*1000)
end_time = int((datetime.strptime("2022-12-31T00:00:00Z", '%Y-%m-%dT%H:%M:%SZ') - datetime(1970,1,1)).total_seconds()*1000)

api_url = f"https://api.binance.com/api/v1/klines?symbol={symbol}&interval={interval}&startTime={start_time}&endTime={end_time}"
response = requests.get(api_url)
if response.status_code == 200:
    data = json.loads(response.text)
    close_prices = [float(candle[4]) for candle in data]
    open_prices = [float(candle[1]) for candle in data]
    high_prices = [float(candle[2]) for candle in data]
    low_prices = [float(candle[3]) for candle in data]
    volume = [float(candle[5]) for candle in data]
else:
    print(f'Error {response.status_code}: {response.text}')
    
# Escalar precios de cierre, abiertos, altos, bajos y volumen y convertirlos en una matriz
scaler = MinMaxScaler(feature_range=(0, 1))
if 'close_prices' in locals():
    close_prices = scaler.fit_transform(np.array(close_prices).reshape(-1, 1))
    open_prices = scaler.fit_transform(np.array(open_prices).reshape(-1, 1))
    high_prices = scaler.fit_transform(np.array(high_prices).reshape(-1, 1))
    low_prices = scaler.fit_transform(np.array(low_prices).reshape(-1, 1))
    volume = scaler.fit_transform(np.array(volume).reshape(-1, 1))
else:
    print("La variable close_prices no ha sido asignada")

#Crear una matriz con todos los datos escalados
prices = np.hstack((close_prices, open_prices, high_prices, low_prices, volume))

#Dividir en entrenamiento y prueba
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size

# Crear matrices de datos con ventanas deslizantes
def create_dataset(open_prices, high_prices, low_prices, close_prices, volume, window_size = 7):
data_X = []
for i in range(len(close_prices) - window_size - 1):
    a = np.column_stack((open_prices[i:(i + window_size)], high_prices[i:(i + window_size)],
    low_prices[i:(i + window_size)], close_prices[i:(i + window_size)], volume[i:(i + window_size)]))
    data_X.append(a)
return np.array(data_X)

train_X = create_dataset(open_prices[:train_size], high_prices[:train_size], low_prices[:train_size], close_prices[:train_size], volume[:train_size])
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 5))

test_X = create_dataset(open_prices[train_size:], high_prices[train_size:], low_prices[train_size:], close_prices[train_size:], volume[train_size:])
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 5))

#Crear las variables de salida para entrenamiento y prueba
train_Y = close_prices[window_size+1:train_size+window_size+1]
test_Y = close_prices[train_size+window_size+1:]

#Crear y entrenar la red neuronal
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(window_size, 5)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_X, train_Y, epochs=100, batch_size=1, verbose=2, validation_data=(test_X, test_Y))

#Hacer predicciones
train_predict = model.predict(train_X)
test_predict = model.predict(test_X)

#Desescalar precios predecidos
train_predict = scaler.inverse_transform(train_predict)
train_Y = scaler.inverse_transform(np.array(train_Y).reshape(-1, 1))
test_predict = scaler.inverse_transform(test_predict)
test_Y = scaler.inverse_transform(np.array(test_Y).reshape(-1, 1))

#Calcular MAE
train_mae = np.mean(np.abs(train_predict - train_Y))
test_mae = np.mean(np.abs(test_predict - test_Y))
print(f'Train MAE: {train_mae}')
print(f'Test MAE: {test_mae}')

#Imprimir las predicciones
prediction = test_predict[-1]
accuracy = (1 - test_mae) * 100
print(f'Predicción para el próximo precio: {prediction}')
print(f'Precisión: {accuracy}%')

#Graficar Resultados
import matplotlib.pyplot as plt
plt.plot(train_Y, label='Precio real')
plt.plot(train_predict, label='Predicción entrenamiento')
plt.plot(test_predict, label='Predicción prueba')
plt.legend()
plt.show()